### santander

In [47]:
import pandas as pd

# Load the Excel file
file_path = 'updated_006p.xlsx'  # Replace with your file path
excel_data = pd.ExcelFile(file_path)


In [48]:

# Load the relevant sheets
pakiet_df = excel_data.parse('pakiet')
bc_df = excel_data.parse('kursy')


In [49]:
# Ensure 'Data księgowania' columns are of the same type (datetime)
pakiet_df['Data księgowania'] = pd.to_datetime(pakiet_df['Data księgowania'], errors='coerce').dt.date
bc_df['Data księgowania'] = pd.to_datetime(bc_df['Data księgowania'], errors='coerce').dt.date

bc_df['Kwota (PLN)'] = -bc_df['Kwota (PLN)']

In [ ]:
pakiet_df.dtypes

In [ ]:
bc_df.dtypes

In [ ]:

# Debug: Check for potential matches between pakiet and bc_valid
print("Potential matches between 'pakiet' and 'bc_valid':")
matches = pakiet_df.merge(
    bc_valid[['Data księgowania', 'Kwota']],
    on=['Data księgowania', 'Kwota'],
    how='inner'
)
print(matches)


In [ ]:

# Check for duplicates in 'bc' based on 'Data księgowania' and 'Kwota'
bc_duplicates = bc_df.duplicated(subset=['Data księgowania', 'Kwota'], keep=False)
bc_duplicates


In [53]:

# Filter valid rows (non-duplicates) in 'bc' sheet
bc_valid = bc_df[~bc_duplicates]

# Debug: Check filtered valid rows
print("Filtered valid rows from 'bc':")
print(bc_valid)


In [ ]:

# Debug: Check data types in both DataFrames
print("Data types in 'pakiet':")
print(pakiet_df.dtypes)
print("\nData types in 'bc_valid':")
print(bc_valid.dtypes)

# Debug: Check unique dates in both DataFrames
print("Unique 'Data księgowania' in 'pakiet':", pakiet_df['Data księgowania'].unique())
print("Unique 'Data księgowania' in 'bc_valid':", bc_valid['Data księgowania'].unique())

# Debug: Check rows in 'pakiet' that didn't merge successfully
unmatched = pakiet_df[~pakiet_df['Data księgowania'].isin(bc_valid['Data księgowania'])]
print("Rows in 'pakiet' with no matching 'Data księgowania' in 'bc_valid':")
print(unmatched)

unmatched_kwota = pakiet_df[~pakiet_df['Kwota'].isin(bc_valid['Kwota'])]
print("Rows in 'pakiet' with no matching 'Kwota' in 'bc_valid':")
print(unmatched_kwota)
print("Unique 'Data księgowania' in 'bc_valid':", bc_valid['Data księgowania'].unique())


In [56]:

# Merge 'pakiet' with valid 'bc' rows based on 'Data księgowania' and 'Kwota', and add 'Kwota (PLN)' column to pakiet from bc's 'kwota_pln)'
# merged = pakiet_df.merge(
#     bc_valid[['Data księgowania', 'Kwota', 'Kwota (PLN)']],
#     on=['Data księgowania', 'Kwota'],
#     how='left',
#     suffixes=('', '_bc')
# )

merged = pakiet_df.merge(
    bc_valid[['Data księgowania', 'Kwota', 'Kwota (PLN)']],
    on=['Data księgowania', 'Kwota'],
    how='left',
    suffixes=('', '_bc')
)



In [ ]:
merged

In [ ]:

# Debug: Check the merged result
print("Merged DataFrame:")
print(merged.head())

# Update the specified columns in 'pakiet' from 'bc'
pakiet_df['Kwota (PLN)'] = merged['Kwota (PLN)_bc']

# Debug: Check if columns were updated
print("Updated 'pakiet' DataFrame:")
print(pakiet_df.head())

# Save the updated 'pakiet' sheet to a new Excel file
output_file_path = 'updated_006p3.xlsx'  # Replace with your desired output file path
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    pakiet_df.to_excel(writer, sheet_name='pakiet', index=False)
    bc_df.to_excel(writer, sheet_name='bc', index=False)

print(f"Updated file saved to {output_file_path}")


### BNP


In [31]:
import pandas as pd
file = '005.xlsx'
excel_data = pd.ExcelFile(file)

In [28]:
# close the file
excel_data.close()

In [32]:
pakiet = excel_data.parse('pakiet')
bc = excel_data.parse('bc')

In [33]:
pakiet['Data księgowania'] = pd.to_datetime(pakiet['Data księgowania'], errors='coerce').dt.date
bc['Data księgowania'] = pd.to_datetime(bc['Data księgowania'], errors='coerce').dt.date

In [34]:
bc['Kwota'] = -bc['Kwota']
bc['Kwota (PLN)'] = -bc['Kwota (PLN)']

In [ ]:
pakiet

In [ ]:
bc.dtypes

In [37]:
bc_duplicates = bc.duplicated(subset=['Data księgowania', 'Kwota'], keep=False)
bc_valid = bc[~bc_duplicates]

In [ ]:
bc_valid

In [39]:
merged = pakiet.merge(
    bc_valid[['Data księgowania', 'Typ dokumentu', 'Typ konta', 'Nr konta', 'Kwota', 'Kwota (PLN)']],
                on=['Data księgowania', 'Kwota'],
                how='left',
                suffixes=('', '_bc')
)

In [40]:
merged['Typ dokumentu'] = merged['Typ dokumentu_bc']
merged['Typ konta'] = merged['Typ konta_bc']
merged['Nr konta'] = merged['Nr konta_bc']
merged['kwota pln ta sama'] = merged['Kwota (PLN)'] == merged['Kwota (PLN)_bc']

In [ ]:
merged

In [42]:
# drop the columns that were added from bc
merged.drop(columns=['Typ dokumentu_bc', 'Typ konta_bc', 'Nr konta_bc'], inplace=True)

In [43]:
# save to excel
output_file_path = 'updated_005.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    pakiet.to_excel(writer, sheet_name='pakiet', index=False)
    bc.to_excel(writer, sheet_name='bc', index=False)
    merged.to_excel(writer, sheet_name='merged', index=False)

In [44]:
# close
excel_data.close()